## Allowable Emissions, Emissions Growth Rates and Engineered CDR

- Allowable emissions account for future ocean and land sinks as estimated by ESMs, and thus allowable emissions can be used as proxy for climate goals. 
- Can we get to net allowable with reasonable mitigation rates and/or reasonably expected growth rates for CDR implementation?
- How many Direct Air Capture (DAC) plants are implied? 
    
For EESC4020 Humans and the Carbon Cycle, Columbia University, Fall 2024 \
    - Developed by G.A. McKinley. Oct/Nov 2024

## Options to be explored by Students
1. Mitigation (section 2b)
     - Is the rate of mitigation implementation linear (GtC/yr) or exponential (%/yr)? 
     - How fast is mitigation implemented?
2. CDR (section 4)
    - Is the rate of CDR implementation linear (GtC/yr) or exponential (%/yr)?
    - How fast is CDR implemented? 

In [ ]:
import pandas as pd
import xarray as xr
import numpy as np

import matplotlib.pyplot as plt
%matplotlib widget 
# ^ allows zooming in to the plots!

import cmocean as cm    

import gcsfs
fs = gcsfs.GCSFileSystem()

%run plotting_functions.ipynb

## 0. Start and end years

In [ ]:
start_year = 2023
end_year = 2075

## 1. Inputs, GCB 2023 (Friedlingstein et al. 2023)

### Current emission rate 2023, in GtC/yr (Table 7) 

In [ ]:
# GtC/yr
rate_emission_2023 = 11.1

### Allowable Emissions Threshholds at 50% likelihood, in GtC (GCB2023, Section 4)

In [ ]:
#GtC
allowable_1p5C = 75
allowable_1p7C = 175
allowable_2C = 315

## 2. Project emissions with historical growth rate and new growth rate reflecting mitigation

### 2a. Recent historical growth rate

In [ ]:
# 2022 and 2023 estimates growth rate of FF emissions, in % / yr  and  LUC emissions approximately constant (GCB 2023)
historical_growth = 1 

In [ ]:
# Initialize the emission dictionary 
emission_historical_rate = {2023: rate_emission_2023}  
emission_historical_cum={2023 : rate_emission_2023}
# Calculate emissions for each year
for year in range(start_year + 1, end_year + 1):
    emission_historical_rate[year] = emission_historical_rate[year - 1] * (1+ historical_growth/100)
    emission_historical_cum[year] = emission_historical_cum[year-1] + emission_historical_rate[year]

In [ ]:
# In what year do the allowable targets get crossed with historical growth rate
print(f"With emissions growing at {historical_growth} %/yr")

value_to_find = allowable_1p5C
key = next((k for k, v in emission_historical_cum.items() if v >= value_to_find and v < value_to_find+20 ), None)
if key is not None:
    print(f"Cumulative emissions exceed {value_to_find} GtC in {key}")
else:
    print(f"Cumulative emissions {value_to_find} GtC not found; 50% likelihood of staying below 1.5C")

value_to_find = allowable_1p7C
key = next((k for k, v in emission_historical_cum.items() if v >= value_to_find and v < value_to_find+20 ), None)
if key is not None:
    print(f"Cumulative emissions exceed {value_to_find} GtC in {key}")
else:
    print(f"Cumulative emissions {value_to_find} GtC not found; 50% likelihood of staying below 1.7C")

value_to_find = allowable_2C
key = next((k for k, v in emission_historical_cum.items() if v >= value_to_find and v < value_to_find+20 ), None)
if key is not None:
    print(f"Cumulative emissions exceed {value_to_find} GtC in {key}")
else:
    print(f"Cumulative emissions {value_to_find} GtC not found; 50% likelihood of staying below 2C")

### 2b. Project emissions to 2050 with modified growth rate
Select Linear or Exponential and the speed of implementation

In [ ]:
##### MAKE A SELECTION ##### 
# how to achieve reductions? 
# In %/yr (linear =0)  or linear (linear = 1)? 
linear = 0
##### MAKE A SELECTION ##### 

In [ ]:
##### MAKE A SELECTION ##### 
# For the choice made in the cell above, chose speed of implementation
############################
# How fast to mitigate emissions? 
# A negative rate indicates declining emissions
if linear ==0: 
    new_growth = -0 # in %/yr
# OR use a linear reduction in GtC/yr; capped to prevent values less than zero
if linear ==1:
    linear_cut = -0 # in GtC/yr
##### MAKE A SELECTION ##### 

In [ ]:
# Initialize the emission dictionary 
emission_new_rate = {2023: rate_emission_2023}  
emission_new_cum={2023 : rate_emission_2023}

if linear ==0: 
    # Calculate emissions for each year
    for year in range(start_year + 1, end_year + 1):
        emission_new_rate[year] = emission_new_rate[year - 1] * (1+ new_growth/100)
        emission_new_cum[year] = emission_new_cum[year-1] + emission_new_rate[year]
if linear ==1: 
        # Initialize the emission dictionary 
        emission_new_rate = {2023: rate_emission_2023}  
        emission_new_cum={2023 : rate_emission_2023}
        # Calculate emissions for each year
        for year in range(start_year + 1, end_year + 1):
            emission_new_rate[year] = emission_new_rate[year - 1] + linear_cut
            # Here, values are set to be no less than zero 
            if emission_new_rate[year] <= 0:
                emission_new_rate[year]=0
            emission_new_cum[year] = emission_new_cum[year-1] + emission_new_rate[year]

## 3. Plot and compare emissions, consider net zero and ability to achieve temperature targets for mitigation case 

### 3a. Plot emission and find net zero emissions year

In [ ]:
# Plot emissions rate
years = list(emission_historical_rate.keys())
emission_historical_rate_plt = np.array(list(emission_historical_rate.values()))
emission_new_rate_plt = np.array(list(emission_new_rate.values()))

plt.figure(figsize=(10, 6))
plt.xlabel('year')
plt.ylabel('Emissions (GtC/yr)')
plt.title('Emissions')
plt.plot(years, emission_historical_rate_plt, color='r', label=('Emissions with historical growth rate='+str(historical_growth)+'%'))
if linear ==0: 
    plt.plot(years, emission_new_rate_plt, color='b', label=('Emissions with new growth rate='+str(new_growth)+'%'))
if linear ==1: 
    plt.plot(years, emission_new_rate_plt, color='b', label=('Emissions with linear cut ='+str(linear_cut)+'GtC/yr'))
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Find net zero emissions year 
# Use 1 GtC/yr as "net zero" since a percent reduction will never equal to zero
#
if linear ==0:
    print(f"With emissions mitigation at {new_growth} %/yr ")
if linear ==1:
    print(f"With emissions mitigation at {linear_cut} GtC/yr ")

value_to_find = 1
key = next((k for k, v in emission_new_rate.items() if v < value_to_find ), None)
if key is not None:
    print(f"Emissions are less than {value_to_find} GtC/yr (~net zero) in {key}")
else:
    print(f"Emissions less than {value_to_find} GtC/yr not found before {end_year}")

In [ ]:
#### USE THIS TO COMPARE TO YOUR CARBON FOOTPRINT in PS5 ####
# SELECT A YEAR TO OUTPUT EMISSIONS RATE, and calculate the percent cut by that year
query_year = 2030
if linear ==0:
    print(f"With mitigation at {new_growth} %/yr ")
if linear ==1:
    print(f"With mitigation at {linear_cut} GtC/yr ")
print(f"Emissions are {round(emission_new_rate[query_year],1)} GtC/yr in {query_year} ")
print(f"Emissions in {query_year} are {round(100*emission_new_rate[query_year]/emission_new_rate[2023],1)} % of 2023 emissions; a {100-round(100*emission_new_rate[query_year]/emission_new_rate[2023],1)}% reduction")
#### USE THIS TO COMPARE TO YOUR CARBON FOOTPRINT in PS5 ####

### 3b. Plot cumulative emissions and assess ability to meet temperature targets based on allowable emissions

In [ ]:
# Plot cumulative emissions
years = list(emission_historical_rate.keys())
emission_historical_cum_plt = np.array(list(emission_historical_cum.values()))
emission_new_cum_plt = np.array(list(emission_new_cum.values()))

plt.figure(figsize=(10, 6))
plt.xlabel('year')
plt.ylabel('Emissions (GtC)')
plt.title('Cumulative Emissions')
plt.plot(years, emission_historical_cum_plt, color='r', label=('Emissions with historical growth rate='+str(historical_growth)+'%'))
if linear ==0: 
    plt.plot(years, emission_new_cum_plt, color='b', label=('Emissions with new growth rate='+str(new_growth)+'%'))
if linear ==1: 
    plt.plot(years, emission_new_cum_plt, color='b', label=('Emissions with linear cut ='+str(linear_cut)+'GtC/yr'))
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
if linear ==0:
    print(f"With emissions mitigation at {new_growth} %/yr ")
if linear ==1:
    print(f"With emissions mitigation at {linear_cut} GtC/yr ")

value_to_find = allowable_1p5C
key = next((k for k, v in emission_new_cum.items() if v >= value_to_find and v < value_to_find+20 ), None)
if key is not None:
    print(f"Cumulative emissions exceed {value_to_find} GtC in {key}")
else:
    print(f"Cumulative emissions {value_to_find} GtC not found; 50% likelihood of staying below 1.5C")

value_to_find = allowable_1p7C
key = next((k for k, v in emission_new_cum.items() if v >= value_to_find and v < value_to_find+20 ), None)
if key is not None:
    print(f"Cumulative emissions exceed {value_to_find} GtC in {key}")
else:
    print(f"Cumulative emissions {value_to_find} GtC not found; 50% likelihood of staying below 1.7C")

value_to_find = allowable_2C
key = next((k for k, v in emission_new_cum.items() if v >= value_to_find and v < value_to_find+20 ), None)
if key is not None:
    print(f"Cumulative emissions exceed {value_to_find} GtC in {key}")
else:
    print(f"Cumulative emissions {value_to_find} GtC not found; 50% likelihood of staying below 2C")

## 4. Add engineered CDR to projections
Select Linear or Exponential and the speed of implementation

In [ ]:
# GCB section 3.3 
# -0.002 MtC/yr DACCS + -0.001 MtC/yr Enhanced Weathering
current_engineered_CDR = -0.003/1000 # in GtC/yr

In [ ]:
##### MAKE A SELECTION ##### 
# how to achieve increase? 
# In %/yr (linear_CDR =0)  or linear (linear_CDR = 1)? 
linear_CDR = 1
##### MAKE A SELECTION ##### 

In [ ]:
##### MAKE A SELECTION ##### 
# For the choice made in the cell above, chose speed of implementation
############################
# How fast to mitigate emissions? 
# A positive rate indicates increasing capacity
if linear_CDR ==0: 
    CDR_growth_rate = 0 # in %/yr
# OR use a linear increase in GtC/yr 
if linear_CDR ==1:
    CDR_growth_linear = 0 # in GtC/yr 
##### MAKE A SELECTION ##### 

In [ ]:
# Initialize the CDR dictionary
engineered_CDR_rate = {2023: current_engineered_CDR}  
engineered_CDR_cum = {2023: current_engineered_CDR}  

if linear_CDR == 0:
    # Calculate CDR for each year
    for year in range(start_year + 1, end_year + 1):
        engineered_CDR_rate[year] = engineered_CDR_rate[year - 1] * (1+ CDR_growth_rate/100)
        engineered_CDR_cum[year] = engineered_CDR_cum[year-1] + engineered_CDR_rate[year]
if linear_CDR == 1:
    # Calculate CDR for each year
    for year in range(start_year + 1, end_year + 1):
        engineered_CDR_rate[year] = engineered_CDR_rate[year - 1] + CDR_growth_linear
        engineered_CDR_cum[year] = engineered_CDR_cum[year-1] + engineered_CDR_rate[year]

In [ ]:
# Add CDR to both scenarios to get effective emissions after CDR
# Initialize 
emission_historical_rate_CDR = {2023: rate_emission_2023}  
emission_new_rate_CDR={2023 : rate_emission_2023}
emission_historical_CDR_cum = {2023: rate_emission_2023}  
emission_new_CDR_cum={2023 : rate_emission_2023}
# Calculate effective emissions for each year
for year in range(start_year + 1, end_year + 1):
    # Add CDR to emissions
    emission_historical_rate_CDR[year] = emission_historical_rate[year] + engineered_CDR_rate[year] 
    emission_new_rate_CDR[year] = emission_new_rate[year] + engineered_CDR_rate[year] 
    # Cumulative
    emission_historical_CDR_cum[year] = emission_historical_CDR_cum[year-1] + emission_historical_rate_CDR[year]
    emission_new_CDR_cum[year] = emission_new_CDR_cum[year-1] + emission_new_rate_CDR[year]

### 4a. Plot emissions + engineered CDR and find net zero emissions year

In [ ]:
years = list(emission_historical_rate.keys())

# extract data to plot
emission_historical_rate_plt = np.array(list(emission_historical_rate.values()))
emission_new_rate_plt = np.array(list(emission_new_rate.values()))
engineered_CDR_rate_plt = np.array(list(engineered_CDR_rate.values()))
emission_historical_rate_plt_CDR = np.array(list(emission_historical_rate_CDR.values()))
emission_new_rate_plt_CDR = np.array(list(emission_new_rate_CDR.values()))

plt.figure(figsize=(10, 6))
plt.xlabel('year')
plt.ylabel('Emissions / Removals (GtC/yr)')
plt.title('Emissions and CDR')
plt.plot(years, emission_historical_rate_plt, color='r', label=('Emissions with historical growth rate='+str(historical_growth)+'%/yr'))

if linear ==0: 
    plt.plot(years, emission_new_rate_plt, color='b', label=('Emissions with new growth rate='+str(new_growth)+'%'))
if linear ==1: 
    plt.plot(years, emission_new_rate_plt, color='b', label=('Emissions with linear cut ='+str(linear_cut)+'GtC/yr'))
    
if linear_CDR==0: 
    plt.plot(years, engineered_CDR_rate_plt, color='k', label=('CDR, growing '+str(CDR_growth_rate)+'%/yr'))
if linear_CDR==1: 
    plt.plot(years, engineered_CDR_rate_plt, color='k', label=('CDR, growing '+str(CDR_growth_linear)+' GtC/yr'))
    
plt.plot(years, emission_historical_rate_plt_CDR, color='r',linestyle='--', label=('Emissions with historical growth rate='+str(historical_growth)+'%/yr'+' + CDR'))
plt.plot(years, emission_new_rate_plt_CDR , color='b',linestyle='--', label=('Emissions with revised emissions + CDR'))
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Find net zero emissions year 
# Use 1 GtC/yr as "net zero" since a percent reduction will never equal to zero
if linear ==0:
    print(f"With emissions mitigation at {new_growth} %/yr ")
if linear ==1:
    print(f"With emissions mitigation at {linear_cut} GtC/yr ")

if linear_CDR ==0:
    print(f"and CDR growing at {CDR_growth_rate} %/yr ")
if linear_CDR ==1:
    print(f"and CDR growing at {CDR_growth_linear} GtC/yr ")

value_to_find = 1
key = next((k for k, v in emission_new_rate_CDR.items() if v < value_to_find ), None)
if key is not None:
    print(f"Net emissions are less than {value_to_find} GtC/yr (~ net zero) in {key}")
else:
    print(f"Net emissions less than {value_to_find} GtC/yr not found before {end_year}")

### 4b. Plot cumulative emissions + engineered CDR

In [ ]:
# Plot cumulative emissions
years = list(emission_historical_rate.keys())
emission_historical_CDR_cum_plt = np.array(list(emission_historical_CDR_cum.values()))
emission_new_CDR_cum_plt = np.array(list(emission_new_CDR_cum.values()))
engineered_CDR_cum_plt = np.array(list(engineered_CDR_cum.values()))

plt.figure(figsize=(10, 6))
plt.xlabel('year')
plt.ylabel('Emissions (GtC)')
plt.title('Cumulative Emissions with CDR')

plt.plot(years, emission_historical_CDR_cum_plt, color='r', label=('Emissions with historical growth rate='+str(historical_growth)+'% + CDR'))
if linear ==0: 
    plt.plot(years, emission_new_CDR_cum_plt, color='b', label=('Emissions with new growth rate='+str(new_growth)+'% + CDR'))
if linear ==1: 
    plt.plot(years, emission_new_CDR_cum_plt, color='b', label=('Emissions with linear cut ='+str(linear_cut)+'GtC/yr+ CDR'))
    
if linear_CDR==0: 
    plt.plot(years, engineered_CDR_cum_plt, color='k', label=('CDR, growing '+str(CDR_growth_rate)+'%/yr'))
if linear_CDR==1: 
    plt.plot(years, engineered_CDR_cum_plt, color='k', label=('CDR, growing '+str(CDR_growth_linear)+' GtC/yr'))
    
plt.legend()
plt.grid(True)
plt.show()

### 4c. Does engineered CDR help achieve Temperature targets? Consider both with historical rate of emissions growth and new rate.

In [ ]:
print(f"With emissions growing at {historical_growth} %/yr")
if linear_CDR ==0:
    print(f"and CDR growing at {CDR_growth_rate} %/yr ")
if linear_CDR ==1:
    print(f"and CDR growing at {CDR_growth_linear} GtC/yr ")

value_to_find = allowable_1p5C
key = next((k for k, v in emission_historical_CDR_cum.items() if v >= value_to_find and v < value_to_find+20 ), None)
if key is not None:
    print(f"Cumulative emissions exceed {value_to_find} GtC in {key}")
else:
    print(f"Cumulative emissions {value_to_find} GtC not found; 50% likelihood of staying below 1.5C")

value_to_find = allowable_1p7C
key = next((k for k, v in emission_historical_CDR_cum.items() if v >= value_to_find and v < value_to_find+20 ), None)
if key is not None:
    print(f"Cumulative emissions exceed {value_to_find} GtC in {key}")
else:
    print(f"Cumulative emissions {value_to_find} GtC not found; 50% likelihood of staying below 1.7C")

value_to_find = allowable_2C
key = next((k for k, v in emission_historical_CDR_cum.items() if v >= value_to_find and v < value_to_find+20 ), None)
if key is not None:
    print(f"Cumulative emissions exceed {value_to_find} GtC in {key}")
else:
    print(f"Cumulative emissions {value_to_find} GtC not found; 50% likelihood of staying below 2C")

In [ ]:
if linear ==0:
    print(f"With emissions mitigation at {new_growth} %/yr ")
if linear ==1:
    print(f"With emissions mitigation at {linear_cut} GtC/yr ")
if linear_CDR ==0:
    print(f"and CDR growing at {CDR_growth_rate} %/yr ")
if linear_CDR ==1:
    print(f"and CDR growing at {CDR_growth_linear} GtC/yr ")


value_to_find = allowable_1p5C
key = next((k for k, v in emission_new_CDR_cum.items() if v >= value_to_find and v < value_to_find+20 ), None)
if key is not None:
    print(f"Cumulative emissions exceed {value_to_find} GtC in {key}")
else:
    print(f"Cumulative emissions {value_to_find} GtC not found; 50% likelihood of staying below 1.5C")

value_to_find = allowable_1p7C
key = next((k for k, v in emission_new_CDR_cum.items() if v >= value_to_find and v < value_to_find+20 ), None)
if key is not None:
    print(f"Cumulative emissions exceed {value_to_find} GtC in {key}")
else:
    print(f"Cumulative emissions {value_to_find} GtC not found; 50% likelihood of staying below 1.7C")

value_to_find = allowable_2C
key = next((k for k, v in emission_new_CDR_cum.items() if v >= value_to_find and v < value_to_find+20 ), None)
if key is not None:
    print(f"Cumulative emissions exceed {value_to_find} GtC in {key}")
else:
    print(f"Cumulative emissions {value_to_find} GtC not found; 50% likelihood of staying below 2C")

### 4d. How much installed capacity of engineered CDR in selected year?

In [ ]:
query_year = 2070
if linear_CDR ==0:
    print(f"With CDR growing at {CDR_growth_rate} %/yr ")
if linear_CDR ==1:
    print(f"With CDR growing at {CDR_growth_linear} GtC/yr ")

print(f"Engineered CDR is {engineered_CDR_rate[query_year]} GtC/yr in {query_year} ")
print(f"Cumulative engineered CDR is {engineered_CDR_cum[query_year]} GtC/yr in {query_year} ")

### 4e. If all engineered CDR is DAC, how many of the currently-largest DAC plants must be installed? 

In [ ]:
# how many of the currently-biggest DAC plants 
biggestDAC = 0.0005 # GtCO2/yr (Stratos plant, Texas, 500,000 tons CO2 / yr, starting mid-2025)
biggestDAC = biggestDAC/3.664 # in GtC/yr
print(f"If all engineered CDR with DAC, this implies {round((engineered_CDR_rate[query_year]*-1)/biggestDAC)} installed plants in {query_year}")